<a href="https://www.kaggle.com/code/isithadinujaya/deep-audio-zoomoing?scriptVersionId=299294570" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Cell 1: Install dependencies (run once)
!pip install gpuRIR soundfile librosa tqdm

# Detect CUDA version and install matching cupy
import subprocess
cuda_version = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
if 'release 12' in cuda_version.stdout:
    !pip install cupy-cuda12x
elif 'release 11' in cuda_version.stdout:
    !pip install cupy-cuda11x
else:
    print("CUDA version not recognised, trying default cupy")
    !pip install cupy

In [ ]:
import os
import random
import json
import numpy as np
import soundfile as sf
import librosa
import pyroomacoustics as pra
from scipy.signal import fftconvolve
from tqdm import tqdm



In [ ]:
pip install pyroomacoustics

In [ ]:
sample_rate = 16000
duration_max = 8

num_mics =8

mic_radius = 0.05

mic_pos = pra.circular_2D_array(center = [0,0], M= num_mics, phi0=0, radius=mic_radius)


room_size = [5,8,3] #width, depth, height 

t_60_range =[0.3,1.3] #reverberation time

SNR_range = [10,40]
Source_dist_min = 0.5  #minimum distance from array





In [ ]:
speech_root = "/kaggle/input/aishell-dataset/wav"

noise_root = "/kaggle/input/datasets/chrisfilo/demand"

output = "/kaggle/working/audi_zoom"

os.makedirs(output, exist_ok = True)

In [ ]:
N_train = 5000
N_Val = 500
test = 100


In [ ]:
def load_speech_files(root):
    "recursively finding all.wav files under root"
    speech_files = []

    for subdir, i, files in os.walk(root):
        for f in files:
            if f.endswith('.wav'):
                speech_files.append(os.path.join(subdir,f))

    return speech_files


def load_noise(root):

    noise_files = []

    for sundir, j, files in os.walk(root):
        for f in files:
            if f.endswith(("wav","flac")):
                noise_files.append(os.path.join(subdir,f))
    return noise_files



In [ ]:
#diiferent room sizes for data set generations


def random_room_params():
    Lx = random.uniform(4,8)
    Ly = random.uniform(4,8)
    Lz = random.uniform(2.5, 4.0)

    room_sz = [Lx,Ly,Lz]
    
    margin = 0.5 #not close to room walls

    array_pos =[random.uniform(margin, Lx - margin),random.uniform(margin, Ly - margin),random.uniform(1,1.5) ]

    return room_sz, array_pos

In [ ]:
def random_source_positions(num_src, room_sz, array_pos, min_dist = Source_dist_min):
    pos = []
    margin = 0.3
    for i in range(num_src):
        while True:
            p = [random.uniform(margin, room_sz[0] - margin),random.uniform(margin, room_sz[1] - margin),random.uniform(1,1.8) ]

            dist = np.linalg.norm(np.array(p) - np.array(array_pos))
        
            if dist >= min_dist :
                pos.append(p)
                break
    return np.array(poss)



In [ ]:
def random_fov():

    az_width = random.uniform(30,180)*np.pi/180
    el_width = random.uniform(10,60)*np.pi/180

    az_center = random.uniform(-np.pi,np.pi)
    el_center = random.uniform(-np.pi/4,np.pi/4)

    az_min = az_center - az_width/2
    az_max = az_center + az_width/2
    el_min = el_center - el_center/2
    el_max = el_center + el_center/2

    return [az_min, az_max], [el_min, el_max]

def in_fov(source_pos, array_pos, az_range, el_range):

    vec = np.array(source_pos) - np.array(array_pos)

    az = np.arctan2(vec[1],vec[0])#azimuth frim x axis

    el = np.arctan2(vec[2], np.sqrt(vec[0]**2 +  vec[1]**2)) #elevations
    


In [ ]:
def build_traget(sources_in_fov, orginal_signals):
    "sum orginal signals in Fov"
    if not source_in_fov:
        return np.zeros_like(orginal_signals[0])
    return np.sum(orginal_signals[idx] for idx in sources_in_fov], axis = 0)

def add_noise(mic_signals, noise_files, snr_db, room_sz, array_pos):
    noise_file  = random.choice(noise_files)
    noise_sig, i = librosa.load(noise_file, sr= sample_rate, mono = True)

    target_len = mic_signals.shape[1]

    if len(noise_sig) < target_len:
        noise_sig = np.tile(noise_sig,int(np.ceil(target_len / len(noise_sig)))+1)

    noise_sig = noise_sig[:target_len]

    margin =0.3
    noise_pos = [random.uniform(margin,room_sz[0] - margin),random.uniform(margin,room_sz[1] - margin), random(0.5,2)]

    #creating a tempory room for noise only
    room_noise = pra.ShoeBox(room_sz, fs = sample_rate, material = pra.Material('hard_surface'), max_order = 15, absorption = None, sigma2_awgn = None)

    mic_array_abs = np.array(mic_pos) + np.array(array_pos[:2])[:,None] #placing microphones at absolute positions

    mic_array_abs = np.vstack((mic_array_abs, np.ones((1,num_mics))*array_pos[2]))  #add z

    room_noise.add_source(noise_pos, signal=noise_sig)
    room_noises.compute_rir()
    room_noise.simulate()
    noise_mic = room_noise.mic_array.signals #shaoe(M,samples)

    min_len = min(noise_mic.shape[1], mic_signals.shape[1])
    noise_mic = noise_mic[:, :min_len]
    mic_signals = mic_signals[: , :min_len]

    #sale_noise power to desired SNR
    clean_power = np.mean(mic_signals[0, :]**2)
    noise_power = np.mean(noise_mic[0, :]**2)

    if noise_power > 0:
        snr_linear = 10**(snr_db /10)
        scale = np.sqrt(clean_power/(snr_linear*noise_power))
        noise_mic *= scale
    else:
        scale = 0

    return mic_signals + noise_mic

